In [1]:
import torch 
from torch import nn

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand((2, 4))
net(X)

tensor([[0.2163],
        [0.2050]], grad_fn=<AddmmBackward>)

In [3]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.1502,  0.0572, -0.2010,  0.1588,  0.2640,  0.2738,  0.1216,  0.3173]])),
             ('bias', tensor([-0.1469]))])

In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1469], requires_grad=True)
tensor([-0.1469])


In [6]:
# 还没有调用反向传播， 所以梯度处于初始状态
print(net[2].weight.grad)

None


In [9]:
# 一次性访问所有参数
print([(name, param.shape) for name, param in net[0].named_parameters()])
print([(name, param.shape) for name, param in net.named_parameters()])

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
[('0.weight', torch.Size([8, 4])), ('0.bias', torch.Size([8])), ('2.weight', torch.Size([1, 8])), ('2.bias', torch.Size([1]))]


In [10]:
net.state_dict()['2.bias'].data

tensor([-0.1469])

In [11]:
net.state_dict()['0.weight'].data

tensor([[ 0.4699,  0.3735,  0.2088,  0.2447],
        [ 0.4568,  0.3108, -0.3175,  0.2233],
        [ 0.4986, -0.0258, -0.2522, -0.1636],
        [ 0.2980, -0.1631, -0.0702,  0.0985],
        [-0.0510,  0.2185,  0.3630,  0.1567],
        [-0.2502,  0.2563,  0.0547,  0.3742],
        [-0.3605, -0.2410,  0.1863, -0.2864],
        [-0.2344,  0.0438,  0.3381, -0.1545]])

In [15]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                        nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}', block1())

    return net

In [16]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2075],
        [-0.2075]], grad_fn=<AddmmBackward>)

In [17]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [18]:
# 访问第一块中第二个子块的第一层的偏置项
rgnet[0][1][0].bias.data

tensor([-0.3823,  0.4014,  0.2415,  0.2122, -0.1248,  0.4835,  0.0740,  0.1361])

### 参数初始化

In [20]:
# 内置初始化
def init_normal(m):
    if type(m) == nn.Linear:
        #   _ 原地操作
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
        
net.apply(init_normal)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [24]:
print(net[0].weight.shape)
print(net[2].weight.shape)

torch.Size([8, 4])
torch.Size([1, 8])


In [25]:
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0017,  0.0089, -0.0080, -0.0030]), tensor(0.))

In [26]:
# 将所有参数初始化为给定的常数
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [28]:
# 对某些块应用不同的初始化方法
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.4313, -0.0091, -0.2203,  0.4924])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [31]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", [(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init ('weight', torch.Size([8, 4]))
Init ('weight', torch.Size([1, 8]))


tensor([[ 0.0000, -7.8727,  0.0000,  5.4798],
        [-0.0000, -7.4710, -6.2498, -6.0562]], grad_fn=<SliceBackward>)

In [32]:
# 还可以直接设置参数
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -6.8727,  1.0000,  6.4798])

In [34]:
# 参数绑定
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                   shared, nn.ReLU(),
                   shared, nn.ReLU(),
                   nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [35]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)